这里提到的占比是指ADC带来的额外电流的平均值占到整体电流消耗的比值。

使用Energy Trace测量ADC带来的额外电流会低于Datasheet的声称值。Energy Trace每秒大约采样800次，相当于采样间隔$1250$μs。这已经远远高于ADC Sample and Hold所需要的时间（大约32cycles到46cycles）。

In [7]:
import yaml
import itertools
import numpy as np
import matplotlib.pyplot as plt


raw_data = yaml.safe_load(open("./rawdata.yaml", "r"))

board_keys = ['MSP430FR5994', 'MSP430FR5969', 'MSP430FR2433']
frequency_keys = ['1MHZ', '2MHZ', '4MHZ']
system_keys = ['idle', 'testbench']
adc_keys = ['ADC_off', 'ADC_on']
interval_keys = ['100us', '150us', '200us', '250us', '300us', '400us', '500us', '600us', '800us', '1000us']

# Take the average value of the current in each case and calculate the amount of additional current brought by the ADC based on it.

for board in board_keys:
    print("[{}]".format(board))
    for frequ, system in itertools.product(frequency_keys, system_keys):
        print("{} {:10}".format(frequ, system), end=': ')
        case_data = raw_data[board][frequ][system]
        adc_off_current = case_data['ADC_off'][0]  
        adc_on_current = np.array([case_data['ADC_on'][int_key][0] for int_key in interval_keys])
        adc_extra_current = adc_on_current - adc_off_current

        proportion = np.array((adc_on_current - adc_off_current) / adc_off_current * 100, dtype=int)
        print(proportion)
    print()


[MSP430FR5994]
1MHZ idle      : [64 40 30 24 21 16 12 11  8  7]
1MHZ testbench : [46 31 23 19 16 12  9  8  6  5]
2MHZ idle      : [50 35 27 22 19 15 13 11  9  8]
2MHZ testbench : [32 22 17 14 11  9  7  6  5  4]
4MHZ idle      : [39 27 20 17 14 11  9  8  7  6]
4MHZ testbench : [21 14 10  8  7  5  4  3  2  2]

[MSP430FR5969]
1MHZ idle      : [68 45 34 28 26 20 17 14 11 10]
1MHZ testbench : [53 35 27 22 18 13 11  9  7  6]
2MHZ idle      : [57 39 30 25 21 17 14 12 10  9]
2MHZ testbench : [37 25 19 16 13 10  8  7  6  5]
4MHZ idle      : [42 27 21 16 14  9  6  5  4  3]
4MHZ testbench : [27 18 14 12 10  8  7  6  6  5]

[MSP430FR2433]
1MHZ idle      : [9 6 5 4 3 3 2 2 2 1]
1MHZ testbench : [8 6 4 4 3 2 2 2 2 1]
2MHZ idle      : [7 4 3 3 2 1 1 1 1 1]
2MHZ testbench : [7 5 4 4 3 3 3 2 2 2]
4MHZ idle      : [7 5 5 4 4 3 3 2 2 2]
4MHZ testbench : [5 4 3 3 3 3 3 3 2 2]



上面的结果展示的是ADC额外电流占开发板本身的电流大小，即假设$I_{\text{sum}} = I_{\text{extra}} + I_{\text{board}}$，那么上面的比值就是$I_{\text{extra}} / I_{\text{board}}$。ADC的间隔依次是`[100us, 150us, 200us, 250us, 300us, 400us, 500us, 600us, 800us, 1000us]`。